### **5.13 - Scalling Airflow with Kubernetes Executors**

After having a quick reminder about Kubernetes, it’s time to discover the Kubernetes Executor. Before  doing so, you have to keep in mind the drawbacks of the Celery Executor to clearly understand why the  Kubernetes Executor has been created. The Celery Executor is  the most common and stable executor used in production since it allows to scale Airflow as much as  we need  just by starting new workers. Nonetheless it brings more complexity and some gotchas.  First, you have to set up extra infrastructure components such as a queue like RabbitMQ or Redis  as well as flower for monitoring Celery.  It means extra work and more maintenance for your team. Also, each time you start a new worker, you have to make  sure that dependencies of your tasks are correctly installed on that new worker.  Otherwise, your tasks will crash with a beautiful error telling you that there is a missing package.  If you have many workers to manage, that can become a bit messy.  The biggest drawback with the Celery Executor is that, when you start a worker, that worker will keep  some resources of your machine whatever the workload. It means that even when you don’t have tasks  to execute anymore, the worker will stay idle wasting resources that could be used for other things.  Lowering your costs and optimizing your resources are very important goals to keep in mind.  Finally, worker nodes are not as resilient as you may think.  Indeed, a task could crash your “airflow worker” process making it unavailable for other tasks.  Nonetheless, don’t think that the Celery Executor is not a good solution for production.  It is still the most widely used and experienced executor.  So now we have seen the drawbacks of the Celery Executor,  we can check the Kubernetes Executor and what it brings.  First as you may guess by its name, it allows to execute your tasks inside a Kubernetes cluster.  Each task runs in a new Pod, so you have one task one Pod.  That’s a great feature as your tasks become totally isolated  and even if a crash happens, only the Pod running where the crash happened is impacted  and it will be automatically restarted by Kubernetes.  This offers you a fine-grained control of your resources in terms of CPU and memory.  If a task needs more memory than CPU, you can specify that in the Pod. Next,  if you remember the biggest issue of the Celery Executor, well the Kubernetes Executor solves  this as it can scale to near-zero if no tasks are being executed.  If no Pods are running, then your resources are released and can be assigned to other things.  Also Pods run to completion, meaning, even if the scheduler goes down, then the tasks will still be able  to complete.  This is particularly useful when we want to upgrade Airflow for example and the scheduler and web server  will be shut down.  Another great feature is that the scheduler subscribes to Kubernetes API event stream.  When a task is finished, Kubernetes gets notified about this event and checkpoints where it is so that  if the scheduler goes down, when it restarts, it goes back to where it was without running anything twice  and avoid forgetting to run some tasks.  Finally, there are three ways of distributing your DAGs inside Kubernetes.  The first one is by using init-containers.  When you start a new Pod, you can run specialized containers that run before app containers in that Pod.  Think of init-containers as a step before running your app containers in a Pod.  Init-containers can contain utilities or setup scripts not present in the app image.  So, you will do a git clone of the repository where your DAGs are and move them into a local  volume before winning Airflow.  The second way to distribute your DAGs is by mounting a shared volume with your DAGs that will be  accessible from your Pods.  Finally, the last way is by packaging your DAGs into a new image that you will deploy using a CI/CD pipeline  into Kubernetes.  I know it may be difficult to follow, but don’t worry we will see an example of that in the next videos.  That being said, I made a little schema to show you from a very high perspective  what Airflow in Kubernetes looks like. So you have all the components, the web server, scheduler and the  metadata database running into distinct Pods.  When a task needs to be triggered, the scheduler sends an event to the kube-api, and then the API starts  a new Pod and executes the command “airflow run” in it for starting the task.  One final note before moving to the drawbacks, by using Kubernetes namespaces,  you can have different instance of Airflow with their own allocated resources inside the same cluster.  A kubernetes namespace are a way to divide cluster resources between multiple users through resource  quotas and limits. By leveraging namespaces,  you could have a dev, pre-prod and prod environment running each an instance of Airflow inside the same  cluster.  Alright. Now what are the drawbacks of using the Kubernetes Executor?  Well, Kubernetes is hard to master.  There are many components to become familiar with, and your DevOps have to be ready for it.  It’s easier if you use a hosted solution but still you have to understand what you do.  So, even if it is less true now, it’s still an experimental executor not as much as used than  the Celery Executor.  All right,  it was quite tough video to follow but stick with me as we will move to practice in the next  video.  